In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import numpy as np
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"
import dgl
import dgl.graphbolt as gb
import torch.nn as nn
import torch.nn.functional as F
import sklearn.metrics
device = torch.device("cpu")

Link Prediction

In [2]:
dataset = gb.BuiltinDataset("cora",root="../../../data/datasets").load()

../../../data/datasets\cora-seeds.zip:   0%|          | 0.00/242k [00:00<?, ?B/s]

Extracting file to ../../../data/datasets
Start to preprocess the on-disk dataset.
Finish preprocessing the on-disk dataset.


In [4]:
graph = dataset.graph
features = dataset.feature 
train_set= dataset.tasks[1].train_set
validation_set= dataset.tasks[1].validation_set
test_set= dataset.tasks[1].test_set
task_name = dataset.tasks[1].metadata["name"]
print(f"task name is {task_name}")


task name is link_prediction
